In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
import torch
from segmentation_pipeline import *

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [2]:
config = load_module("../configs/diffusion_hippocampus.py")
variables = dict(DATASET_PATH="X:/Datasets/Diffusion_MRI_cropped/", CHECKPOINTS_PATH="X:/Checkpoints/")
context = config.get_context(device, variables)
context.init_components()

In [ ]:
import copy
import torchio as tio
from pathlib import Path
import shutil

out_path = Path("X:\\Datasets\\Diffusion_MRI_cropped\\")

crop_transform = tio.CropOrPad((96, 88, 24), padding_mode='minimum', mask_name='whole_roi_union')

for i, subject in enumerate(context.dataset.subjects):
    subject = copy.deepcopy(subject)
    subject.load()
    subject = crop_transform(subject)
    
    subject_path = Path(subject.folder)
    out_subject_path = out_path / "subjects" / subject['name']
    out_subject_path.mkdir(exist_ok=True)

    for image_name, image in subject.get_images_dict(intensity_only=False).items():
        if "uniform" in image and image['uniform']:
            continue
        image.save(out_subject_path / f"{image_name}.nii.gz")
    shutil.copy(subject_path / "attributes.json", out_subject_path / "attributes.json")

    if i == 0:
        uniform_path = out_path / "atlas"
        uniform_path.mkdir(exist_ok=True)
        for image_name, image in subject.get_images_dict(intensity_only=False).items():
            if "uniform" not in image or not image['uniform']:
                continue
            image.save(uniform_path / f"{image_name}.nii.gz")
            
    if i % 10 == 0:
        print(f"Saved subject {subject.name} {i}/{len(context.dataset)}")


In [ ]:
shape = context.dataset.subjects[0].shape
all_hippo = torch.zeros(size=shape, dtype=torch.int64)
N = 0

for subject in context.dataset.subjects:
    if 'whole_roi' in subject:
        N += 1
        all_hippo += (subject['whole_roi'].data > 0).int()

In [ ]:
import torchio as tio
from pathlib import Path

whole_roi_union = tio.LabelMap(tensor=(all_hippo != 0))
whole_roi_mean = tio.LabelMap(tensor=(all_hippo / N) > 0.5)

whole_roi_union.save(Path(context.dataset.root) / "atlas" / "whole_roi_union.nii.gz")
whole_roi_mean.save(Path(context.dataset.root) / "atlas" / "whole_roi_mean.nii.gz")

In [ ]:
from segmentation_pipeline.data_processing.dataset_fingerprint import *

get_bounds(whole_roi_union['data'][0])

In [ ]:
subject_fingerprints, summary_fingerprint = get_dataset_fingerprint(
    context.dataset,
    transform=EnforceConsistentAffine(),
    save=True
)

In [ ]:
import json
from pathlib import Path

with (Path(context.dataset.root) / "fingerprint" / "subject_fingerprints.json").open() as f:
    subject_fingerprints = json.load(f)
    
with (Path(context.dataset.root) / "fingerprint" / "fingerprint.json").open() as f:
    fingerprint = json.load(f)

In [ ]:
import numpy as np

print(fingerprint['label_crop']['whole_roi']['all'])

crop_mean = np.array(fingerprint['label_crop']['whole_roi']['all']['median'])
crop_std = np.array(fingerprint['label_crop']['whole_roi']['all']['std'])

print(crop_mean)
print(crop_std)

for subject_name, fp in subject_fingerprints.items():
    try:
        subject_crop = np.array(fp['label_crop']['whole_roi']['all'])
    except:
        continue
    diff = np.abs(crop_mean - subject_crop)
    if np.any(diff > 3 * crop_std):
        print(subject_name, subject_crop, diff / crop_std)

In [3]:
from pathlib import Path
import torchio as tio
import shutil

dataset = context.dataset.get_cohort_dataset("cbbrain")
subject_paths = {subject['subject_id']: subject['folder'] for subject in dataset.subjects}

def get_subject_id(file_path):
    out = file_path.stem
    out = out.split(".")[0]
    out = out.split("_")[2]
    out = int(out)
    return out

"""
full_dwi_path = Path("X:\\Datasets\\Diffusion_MRI_extra\\Diffusion_MRI_old\\Cb_Brain_n153\\Full")
for full_dwi_path in full_dwi_path.iterdir():
    subject_id = get_subject_id(full_dwi_path)
    out_path = Path(subject_paths[subject_id])
    
    full_dwi = tio.ScalarImage(full_dwi_path, channels_last=True)
    full_dwi.load()
    print(f"Saving {full_dwi_path} to {out_path}")
    full_dwi.save(out_path / "full_dwi.nii.gz")
"""

bvec_folder_path = Path("X:\\Datasets\\Diffusion_MRI_extra\\Diffusion_MRI_old\\Cb_Brain_n153\\Gradient_Info")
for bvec_path in bvec_folder_path.iterdir():
    subject_id = get_subject_id(bvec_path)
    out_path = Path(subject_paths[subject_id])
    shutil.copyfile(bvec_path, out_path / "full_dwi_grad.b")
    

In [ ]:
from pathlib import Path
import shutil

dest_path = Path("X:\\Predictions\\Diffusion_MRI\\isbi_models")

for subject in context.dataset.subjects:
    subject_path = Path(subject['folder'])
    
    dest_subject_path = dest_path / subject['name']
    dest_subject_path.mkdir(exist_ok=True)
    
    for subject_file in subject_path.iterdir():
        if "pred" in subject_file.name:
            shutil.move(subject_file, dest_subject_path / subject_file.name)